# United Kingdom

The code is pretty ugly... parsing excel files that change between years.

In [ ]:
# default_exp countries.uk

In [ ]:
#export
from weekly_mort.imports import *
from weekly_mort.core import *

import openpyxl

## Settings

In [ ]:
down_dir, proc_dir = create_country_dirs('United Kingdom')

In [ ]:
#export
LAST_MODIFIED = datetime.date.today()

Get URL for latest excel sheet.

In [ ]:
for i in range(datetime.date.today().isocalendar()[1]+1, 10, -1):
    url2020 = f'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2f' \
              f'birthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisional' \
              f'figuresondeathsregisteredinenglandandwales%2f2020/publishedweek{i}2020.xlsx'
    out = requests.get(url2020)
    if out.status_code == 200:
        break

In [ ]:
urls = {2020: url2020,
        2019:'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2019/publishedweek522019.xls',
        2018: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2018/publishedweek522018withupdatedrespiratoryrow.xls',
        2017: 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2017/publishedweek522017.xls'
}

## Process 2020 deaths

In [ ]:
with open(down_dir/'deaths2020.xlsx', 'wb') as f:
    f.write(requests.get(urls[2020]).content)

wb = openpyxl.load_workbook(down_dir/'deaths2020.xlsx')
sheet = wb['Weekly figures 2020']

In [ ]:
def filter_row(row):
    return list(filter(lambda x: x is not None, row[2:]))

def get_row_contents(sheet, first_col_val, colcheck = 0):
    for row in sheet.values:
        if row[colcheck] == first_col_val:
            return filter_row(row)

### Overall Deaths

In [ ]:
enddates = get_row_contents(sheet, 'Week ended')

deaths = get_row_contents(sheet, 'Total deaths, all ages')

resp_deaths = get_row_contents(sheet,
                               'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)',
                               1)

assert len(deaths) == len(resp_deaths)

In [ ]:
total = pd.DataFrame({'Week':[*enddates[:len(deaths)], *enddates[:len(deaths)]],
                      'Condition': [*list(itertools.repeat('Total', len(deaths))),
                                    *list(itertools.repeat('Respiratory', len(deaths)))],
                      'Deaths':[*deaths, *resp_deaths]})
total['Age'], total['Year'], total['Region'], total['Sex'] = 'Total', 2020, 'Total', 'Total'

total.head()

,Week,Condition,Deaths,Age,Year,Region,Sex
0,2020-01-03,Total,12254,Total,2020,Total,Total
1,2020-01-10,Total,14058,Total,2020,Total,Total
2,2020-01-17,Total,12990,Total,2020,Total,Total
3,2020-01-24,Total,11856,Total,2020,Total,Total
4,2020-01-31,Total,11612,Total,2020,Total,Total


### Death by Age

In [ ]:
AGE_LIST = [
    '<1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29',
    '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64',
    '65-69', '70-74', '75-79', '80-84', '85-89', '90+'   
]

In [ ]:
def _process_section(row_gen, row, Sex='Total', year=2020, enddates=enddates):
    dfs = []
    found_flag = False
    while True:
        if row[1] in AGE_LIST:
            found_flag = True
            deaths = filter_row(row)
            df = pd.DataFrame({'Week':enddates[: len(deaths)], 'Deaths': deaths})
            df['Age'], df['Year'], df['Sex'] = row[1], year, Sex
            df['Condition'], df['Region'] = 'Total', 'Total'
            dfs.append(df)
        elif found_flag:
            break
        row = next(row_gen)
    return row, pd.concat(dfs)

In [ ]:
gen_ex = sheet.values

for row in gen_ex:
    if row[1] and ('Persons' in row[1]):
        break

_, out = _process_section(gen_ex, row)

assert len(out) % len(AGE_LIST) == 0

out.head()

,Week,Deaths,Age,Year,Sex,Condition,Region
0,2020-01-03,48,<1,2020,Total,Total,Total
1,2020-01-10,50,<1,2020,Total,Total,Total
2,2020-01-17,69,<1,2020,Total,Total,Total
3,2020-01-24,53,<1,2020,Total,Total,Total
4,2020-01-31,50,<1,2020,Total,Total,Total


In [ ]:
def get_age_breakdown(sheet, year=2020, enddates=enddates):
    row_gen = sheet.values

    dfs = []
    pflag, fflag, mflag = [False]*3
    row = next(row_gen)

    while True:
        if row[1]:
            if 'Persons' in row[1]:
                row, df = _process_section(row_gen, row, 'Total', year, enddates)
                dfs.append(df)
                pflag = True
            if 'Males' in row[1]:
                row, df = _process_section(row_gen, row, 'Male', year, enddates)
                dfs.append(df)
                mflag = True
            if 'Females' in row[1]:
                row, df = _process_section(row_gen, row, 'Female', year, enddates)
                dfs.append(df)
                fflag = True
        elif pflag and mflag and fflag:
            break
        row = next(row_gen)
    return pd.concat(dfs)

In [ ]:
byages = get_age_breakdown(sheet)

In [ ]:
byages.head()

,Week,Deaths,Age,Year,Sex,Condition,Region
0,2020-01-03,48,<1,2020,Total,Total,Total
1,2020-01-10,50,<1,2020,Total,Total,Total
2,2020-01-17,69,<1,2020,Total,Total,Total
3,2020-01-24,53,<1,2020,Total,Total,Total
4,2020-01-31,50,<1,2020,Total,Total,Total


We need to group ages into tighter categories to make it compatible with previous years.

In [ ]:
def _age_map(x):
    if x == '<1':
        return 'Under 1 year'
    elif x in ['1-4', '5-9', '10-14']:
        return '01-14'
    elif x in ['15-19', '20-24', '25-29', '30-34', '35-39', '40-44']:
        return '15-44'
    elif x in ['45-49', '50-54', '55-59', '60-64']:
        return '45-64'
    elif x in ['65-69', '70-74']:
        return '65-74'
    elif x in ['75-79', '80-84']:
        return '75-84'
    elif x in ['85-89', '90+']:
        return '85+'
    else:
        raise ValueError

In [ ]:
byages.Age = byages.Age.apply(_age_map)

In [ ]:
byages = byages.groupby(['Week', 'Year', 'Age', 'Sex', 'Condition', 'Region']).agg(sum).reset_index()

In [ ]:
byages.head()

,Week,Year,Age,Sex,Condition,Region,Deaths
0,2020-01-03,2020,01-14,Female,Total,Total,8
1,2020-01-03,2020,01-14,Male,Total,Total,8
2,2020-01-03,2020,01-14,Total,Total,Total,16
3,2020-01-03,2020,15-44,Female,Total,Total,74
4,2020-01-03,2020,15-44,Male,Total,Total,115


In [ ]:
assert all(byages.Week.value_counts() == 21)

In [ ]:
AGE_LIST = ['Under 1 year', '01-14', '15-44', '45-64', '65-74', '75-84', '85+']

In [ ]:
assert all(byages.Age.isin(AGE_LIST))

### By geography

In [ ]:
def get_geographical(sheet, year=2020, enddates=enddates):

    found_flag = False
    dfs = []

    for row in sheet.values:
        if row[1] and ('Deaths by region of usual residence' in row[1]):
            found_flag = True
            continue
        elif not found_flag:
            continue

        if row[1] is None:
            break
        deaths = filter_row(row)
        df = pd.DataFrame({'Week':enddates[: len(deaths)], 'Deaths': deaths})
        df['Age'], df['Year'], df['Sex'] = 'Total', year, 'Total'
        df['Condition'], df['Region'] = 'Total', row[1]
        dfs.append(df)

    return pd.concat(dfs)

In [ ]:
bygeo = get_geographical(sheet)

In [ ]:
all2020 = pd.concat([total, byages, bygeo], sort=True)

assert not any(all2020.isna().sum())

cols = ['Week', 'Year', 'Age', 'Sex', 'Condition', 'Region', 'Deaths']
all2020 = all2020[cols]

all2020.head()

,Week,Year,Age,Sex,Condition,Region,Deaths
0,2020-01-03,2020,Total,Total,Total,Total,12254
1,2020-01-10,2020,Total,Total,Total,Total,14058
2,2020-01-17,2020,Total,Total,Total,Total,12990
3,2020-01-24,2020,Total,Total,Total,Total,11856
4,2020-01-31,2020,Total,Total,Total,Total,11612


## Process Previous Years

First we convert previous years from `xls` to `xlsx` so we can use the previous approach.

In [ ]:
import xlrd
from openpyxl.workbook import Workbook

def cvt_xls_to_xlsx(src_file_path, dst_file_path):
    '''
    From https://stackoverflow.com/questions/9918646/how-to-convert-xls-to-xlsx/42574983#42574983
    '''
    book_xls = xlrd.open_workbook(src_file_path)
    book_xlsx = Workbook()

    sheet_names = book_xls.sheet_names()
    for sheet_index, sheet_name in enumerate(sheet_names):
        sheet_xls = book_xls.sheet_by_name(sheet_name)
        if sheet_index == 0:
            sheet_xlsx = book_xlsx.active
            sheet_xlsx.title = sheet_name
        else:
            sheet_xlsx = book_xlsx.create_sheet(title=sheet_name)

        for row in range(0, sheet_xls.nrows):
            for col in range(0, sheet_xls.ncols):
                sheet_xlsx.cell(row = row+1 , column = col+1).value = sheet_xls.cell_value(row, col)

    book_xlsx.save(dst_file_path)

In [ ]:
years = list(range(2017, 2020))

In [ ]:
for year in years:
    with open(down_dir/f'deaths{year}.xls', 'wb') as f:
        f.write(requests.get(urls[year]).content)
    
    cvt_xls_to_xlsx(down_dir/f'deaths{year}.xls', down_dir/f'deaths{year}.xlsx')

In [ ]:
wbs = [openpyxl.load_workbook(down_dir/f'deaths{year}.xlsx') for year in years]

sheetnames = [f'Weekly figures {y}' for y in years]

sheets = [wb[shname] for wb, shname in zip(wbs, sheetnames)]

In [ ]:
def conv2date(xldate, datemode=0):
    '''
    from https://stackoverflow.com/questions/1108428/how-do-i-read-a-date-in-excel-format-in-python
    '''
    # datemode: 0 for 1900-based, 1 for 1904-based
    return (
        datetime.datetime(1899, 12, 30)
        + datetime.timedelta(days=xldate + 1462 * datemode)
        )

In [ ]:
_conv_dates = lambda x: [conv2date(y) for y in x]

enddates = [_conv_dates(get_row_contents(sh, 'Week ended')) for sh in sheets]

In [ ]:
byage = [get_age_breakdown(sh, y, end) for sh, y, end in zip(sheets, years, enddates)]

bygeo = [get_geographical(sh, y, end) for sh, y, end in zip(sheets, years, enddates)]

In [ ]:
def get_totals(sheet, year, enddates):
    deaths = get_row_contents(sheet, 'Total deaths, all ages')
    
    resp_deaths = get_row_contents(sheet,
                               'All respiratory diseases (ICD-10 J00-J99)\nICD-10 v 2013 (IRIS)',
                               1)
    
    total = pd.DataFrame({'Week':[*enddates[:len(deaths)], *enddates[:len(deaths)]],
                          'Condition': [*list(itertools.repeat('Total', len(deaths))),
                                        *list(itertools.repeat('Respiratory', len(deaths)))],
                          'Deaths':[*deaths, *resp_deaths]})
    total['Age'], total['Year'], total['Region'], total['Sex'] = 'Total', year, 'Total', 'Total'
    return total

In [ ]:
totals = [get_totals(sh, y, end) for sh, y, end in zip(sheets, years, enddates)]

In [ ]:
all_dfs = [pd.concat([t, a, g], sort=True) for t, a, g in zip(totals, byage, bygeo)]

all_dfs = [df[cols] for df in all_dfs]

## Process all years

In [ ]:
df = pd.concat([all2020] + all_dfs)

Just as for other countries, let's make a seperate mapping from week index to start & end dates.

For the UK, dates end on Fridays, and the first week includes days from the previous year.

In [ ]:
dates = pd.DataFrame(itertools.chain(gen_weekdates(2017, datetime.date(2016, 12, 31), 4),
                                     gen_weekdates(2018, datetime.date(2017, 12, 30), 4),
                                     gen_weekdates(2019, datetime.date(2018, 12, 29), 4),
                                     gen_weekdates(2020, datetime.date(2019, 12, 28), 4)))

dates.columns = ['Year', 'Week', 'Start', 'End']

In [ ]:
_week2idx = {end: week for week, end in zip(dates.Week, dates.End)}

In [ ]:
df['Week'] = df.Week.apply(lambda x: _week2idx[datetime.date(x.year, x.month, x.day)])

In [ ]:
df.head()

,Week,Year,Age,Sex,Condition,Region,Deaths
0,0,2020,Total,Total,Total,Total,12254
1,1,2020,Total,Total,Total,Total,14058
2,2,2020,Total,Total,Total,Total,12990
3,3,2020,Total,Total,Total,Total,11856
4,4,2020,Total,Total,Total,Total,11612


## Save

In [ ]:
df.to_csv(proc_dir/'deaths.csv', index=False)
dates.to_csv(proc_dir/'week_dates.csv', index=False)

## Convert

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 010_UK.ipynb.
Converted 020_Netherlands.ipynb.
Converted 03_collect.ipynb.
Converted index.ipynb.
